In [1]:
# Reference: Extracting Song Data From the Spotify API Using Python
# https://towardsdatascience.com/extracting-song-data-from-the-spotify-api-using-python-b1e79388d50

# Import dependencies
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from config import cid, secret

In [2]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


In [3]:
#uri = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=77d8f5cd51cd478d"

In [4]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [5]:
print(track_uris)

['spotify:track:3nqQXoyQOWXiESFLlDF1hG', 'spotify:track:4uUG5RXrOk84mYEfFvj3cK', 'spotify:track:4Dvkj6JhhA12EX05fT7y2e', 'spotify:track:2tTmW7RDtMQtBk7m2rYeSw', 'spotify:track:5ww2BF9slyYgNOk37BlC4u', 'spotify:track:1IHWl5LamUGEuP4ozKQSXZ', 'spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx', 'spotify:track:5IgjP7X4th6nMNDh4akUHb', 'spotify:track:4h9wh7iOZ0GGn8QVp4RAOB', 'spotify:track:7gRFDGEzF9UkBV233yv2dc', 'spotify:track:4k6Uh1HXdhtusDW5y8Gbvy', 'spotify:track:53tfEupEzQRtVFOeZvk7xq', 'spotify:track:5Eax0qFko2dh7Rl2lYs3bx', 'spotify:track:3k3NWokhRRkEPhCzPmV8TW', 'spotify:track:2yjlYDiNiQkdxVqTlaSrlX', 'spotify:track:5zwwW9Oq7ubSxoCGyW1nbY', 'spotify:track:7jtQIBanIiJOMS6RyCx6jZ', 'spotify:track:6xGruZOHLs39ZbVccQTuPZ', 'spotify:track:6zJejIfVYLgjud3lTk4DLB', 'spotify:track:31i56LZnwE6uSu3exoHjtB', 'spotify:track:2QjOHCTQ1Jl3zawyYOpxh6', 'spotify:track:02MWAaffLxlfxAUY7c5dvx', 'spotify:track:6Xom58OOXk2SoU711L2IXO', 'spotify:track:1xzi1Jcr7mEi9K2RfzLOqS', 'spotify:track:1xK59OXxi2TAAAbmZK0kBL',

In [6]:
# Extract name of each track, main artist, name, popularity, genre, album, popularity of the track
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]


In [18]:
# Extracting Features from Tracks
sp.audio_features(track_uri)[0]

{'danceability': 0.704,
 'energy': 0.756,
 'key': 0,
 'loudness': -4.948,
 'mode': 1,
 'speechiness': 0.448,
 'acousticness': 0.298,
 'instrumentalness': 3.75e-06,
 'liveness': 0.0852,
 'valence': 0.461,
 'tempo': 121.737,
 'type': 'audio_features',
 'id': '6i1g5ZRmJZAkDwBaUZ3f2i',
 'uri': 'spotify:track:6i1g5ZRmJZAkDwBaUZ3f2i',
 'track_href': 'https://api.spotify.com/v1/tracks/6i1g5ZRmJZAkDwBaUZ3f2i',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6i1g5ZRmJZAkDwBaUZ3f2i',
 'duration_ms': 141340,
 'time_signature': 3}

In [9]:
track

{'added_at': '2022-10-07T11:36:02Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
  'href': 'https://api.spotify.com/v1/users/',
  'id': '',
  'type': 'user',
  'uri': 'spotify:user:'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4axKuDPr6WKcDCyh8vueTY'},
     'href': 'https://api.spotify.com/v1/artists/4axKuDPr6WKcDCyh8vueTY',
     'id': '4axKuDPr6WKcDCyh8vueTY',
     'name': 'Luar La L',
     'type': 'artist',
     'uri': 'spotify:artist:4axKuDPr6WKcDCyh8vueTY'}],
   'available_markets': ['AD',
    'AE',
    'AG',
    'AL',
    'AM',
    'AO',
    'AR',
    'AT',
    'AU',
    'AZ',
    'BA',
    'BB',
    'BD',
    'BE',
    'BF',
    'BG',
    'BH',
    'BI',
    'BJ',
    'BN',
    'BO',
    'BR',
    'BS',
    'BT',
    'BW',
    'BY',
    'BZ',
    'CA',
    'CD',
    'CG',
    'CH',
    'CI',
    'CL',
    'CM',
    'CO',
   